In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

import keras
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Activation, Flatten
from keras.callbacks import  LambdaCallback


C:\Users\nbbwang\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X = np.genfromtxt("../data/mnist.data")
y = np.genfromtxt("../data/mnist.labels")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [3]:
# this is ONE set of hyperparameters 
#(e.g., a single element from the list generated by list(ParameterGrid(hyperparams)))
HY = {
    "conv_layers": [(32,(3,3),(2, 2)), (10,(3,3),(2,2))],
    "dense_layers" : [5,3],
    "activation": "relu", 
    "dropout":  0,
    "k_reg": .00001,
    "learning_rate": .01,
    "grad_clip_norm": .5,
    "batch_size": 20}


In [4]:
X_train_c = X_train.reshape(len(X_train), 28, 28, 1)
X_test_c = X_test.reshape(len(X_test), 28, 28, 1)


model = Sequential()

# ADD CONVOLUTION LAYERS
for i,c_params in enumerate(HY["conv_layers"]):
    
    num_filters, kernel_size, pooling_size = c_params
    
    # if it's the first layer, need to specify input shape
    if i==0:
        model.add(Conv2D(num_filters, kernel_size, kernel_regularizer=regularizers.l2(HY["k_reg"]), input_shape=(28,28,1)))
    else:
        model.add(Conv2D(num_filters, kernel_size, kernel_regularizer=regularizers.l2(HY["k_reg"])))
    
    # add activation, pooling, dropout 
    model.add(Activation(HY["activation"]))
    if pooling_size:           
        model.add(MaxPooling2D(pool_size=pooling_size))
    if HY["dropout"]:
        model.add(Dropout(HY["dropout"]))
            
model.add(Flatten())

for i,dense_nodes in enumerate(HY["dense_layers"]):

    model.add(Dense(dense_nodes, kernel_regularizer=regularizers.l2(HY["k_reg"])))
    
    # add activation and dropout 
    model.add(Activation(HY["activation"]))
    if HY["dropout"]:
        model.add(Dropout(HY["dropout"]))


# once all the hidden nodes are added, add the output layer with a softmax activation
model.add(Dense(10))
model.add(Activation('softmax'))


model.compile(optimizer=optimizers.SGD(lr=HY["learning_rate"], clipnorm=HY["grad_clip_norm"]),  
              loss='categorical_crossentropy', metrics=['accuracy'])


def report_ep_loss(ep,logs):
    print(ep, logs['val_loss'])
    #TODO: REPLACE PRINT WITH AN UPDATE TO THE LEADER


history = model.fit(X_train_c, y_train, validation_data=(X_test_c, y_test), epochs=5, verbose=1, 
                    callbacks=[LambdaCallback(on_epoch_end=report_ep_loss)])

# If we want early stopping, add as callback above: 
#keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

# TODO: send trained model to the leader 

Train on 44000 samples, validate on 11000 samples
Epoch 1/5
44000/44000 [==============================] - 33s 741us/step - loss: 2.1570 - acc: 0.1662 - val_loss: 1.9879 - val_acc: 0.2132
0 1.9878532257080077
Epoch 2/5
44000/44000 [==============================] - 30s 691us/step - loss: 1.8802 - acc: 0.2457 - val_loss: 1.7990 - val_acc: 0.2785
1 1.7989718185771595
Epoch 3/5
44000/44000 [==============================] - 30s 687us/step - loss: 1.7467 - acc: 0.3022 - val_loss: 1.6885 - val_acc: 0.3220
2 1.6884956371133977
Epoch 4/5
44000/44000 [==============================] - 30s 686us/step - loss: 1.6313 - acc: 0.3477 - val_loss: 1.5702 - val_acc: 0.3689
3 1.5701594130776144
Epoch 5/5
44000/44000 [==============================] - 30s 692us/step - loss: 1.5098 - acc: 0.4038 - val_loss: 1.4598 - val_acc: 0.4209
4 1.459828434857455


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 10)        2890      
_________________________________________________________________
activation_2 (Activation)    (None, 11, 11, 10)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 10)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 250)               0         
__________